In [1]:
import glob
import os
import sys
import random
import time
import numpy as np

try:
    sys.path.append(glob.glob('carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla


class CarlaAgent:
    actor_list = []
    client = []
    world = []
    BP = []
    vehicle = []
    camera = []
    cameraS = []
    control = []
    image = []
    imageS = []

    def __init__(self):
        # Connect to server
        self.client = carla.Client('localhost', 2000)
        self.client.set_timeout(4.0)
        time.sleep(4.0)
        print("Establishing Connection to Server")

        # Once we have a client we can retrieve the world that is currently
        # running.
        self.world = self.client.load_world('Town01')
        #self.world = self.client.get_world()


        # The world contains the list blueprints that we can use for adding new
        # actors into the simulation.
        self.BP = self.world.get_blueprint_library()

    def spawn_vehicle(self):
        # Now let's filter all the blueprints of type 'vehicle' and choose one
        # at random.
        BP_vehicle = random.choice(self.BP.filter('vehicle'))

        # A blueprint contains the list of attributes that define a vehicle's
        # instance, we can read them and modify some of them. For instance,
        # let's randomize its color.
        if BP_vehicle.has_attribute('color'):
            color = random.choice(BP_vehicle.get_attribute('color').recommended_values)
            BP_vehicle.set_attribute('color', color)

        # Now we need to give an initial transform to the vehicle. We choose a
        # random transform from the list of recommended spawn points of the map.
        transform = random.choice(self.world.get_map().get_spawn_points())

        # So let's tell the world to spawn the vehicle.
        self.vehicle = self.world.spawn_actor(BP_vehicle, transform)
        print(transform)

        # It is important to note that the actors we create won't be destroyed
        # unless we call their "destroy" function. If we fail to call "destroy"
        # they will stay in the simulation even after we quit the Python script.
        # For that reason, we are storing all the actors we create so we can
        # destroy them afterwards.
        self.actor_list.append(self.vehicle)
        print('created %s' % self.vehicle.type_id)

    def attach_camera(self):
        def camera_callback(image):
            array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
            array = np.reshape(array, (image.height, image.width, 4))  # RGBA format
            array = array[:, :, :3]  # Take only RGB
            self.image = array

        # Let's add now a "depth" camera attached to the vehicle. Note that the
        # transform we give here is now relative to the vehicle.
        BP_camera = self.BP.find('sensor.camera.rgb')
        camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
        self.camera = self.world.spawn_actor(BP_camera, camera_transform, attach_to=self.vehicle)
        self.actor_list.append(self.camera)
        print('created %s' % self.camera.type_id)

        # attach camera callback function
        self.camera.listen(lambda image: camera_callback(image))

    def attach_cameraS(self):
        def semantic_callback(image):
            array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
            array = np.reshape(array, (image.height, image.width, 4))  # RGBA format
            array = array[:, :, :3]  # Take only RGB
            self.imageS = array

        # Let's add now a "depth" camera attached to the vehicle. Note that the
        # transform we give here is now relative to the vehicle.
        BP_semantic = self.BP.find('sensor.camera.semantic_segmentation')
        camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
        self.cameraS = self.world.spawn_actor(BP_semantic, camera_transform, attach_to=self.vehicle)
        self.actor_list.append(self.cameraS)
        print('created %s' % self.cameraS.type_id)

        # attach camera callback function
        self.cameraS.listen(lambda image: semantic_callback(image))

    def autopilot(self, t):
        print("Starting autopilot for " + str(t) + " seconds")
        # Let's put the vehicle to drive around.
        self.vehicle.set_autopilot(True)
        # Drive around for 30 second and then stop autopilot
        time.sleep(t)
        self.vehicle.set_autopilot(False)
        print("Stopping autopilot")

    def attach_controller(self):
        self.vehicle.set_autopilot(False)
        self.control = carla.VehicleControl()

    def terminate(self):
        print('destroying actors')
        self.camera.destroy()
        self.client.apply_batch([carla.command.DestroyActor(x) for x in self.actor_list])
        print('terminated')

In [2]:
myAgent = CarlaAgent()

Establishing Connection to Server


## Changing the Map

In [15]:
print(myAgent.client.get_available_maps())

['/Game/Carla/Maps/Town06', '/Game/Carla/Maps/Racetrack03', '/Game/Carla/Maps/Town05', '/Game/Carla/Maps/Town05_Opt', '/Game/Carla/Maps/Town03', '/Game/Carla/Maps/Town07', '/Game/Carla/Maps/Town10HD_Opt', '/Game/Carla/Maps/Town01_Opt', '/Game/Carla/Maps/Racetrack04', '/Game/Carla/Maps/Town10HD', '/Game/Carla/Maps/Town04', '/Game/Carla/Maps/Racetrack01', '/Game/Carla/Maps/Town06_Opt', '/Game/Carla/Maps/Racetrack02', '/Game/Carla/Maps/Town07_Opt', '/Game/Carla/Maps/Town04_Opt', '/Game/Carla/Maps/Town03_Opt', '/Game/Carla/Maps/Town02', '/Game/Carla/Maps/Town01', '/Game/Carla/Maps/Town02_Opt']


In [3]:
myAgent.world = myAgent.client.load_world("Town04")

## Terminate Simulation

In [4]:
myAgent.terminate()

destroying actors


AttributeError: 'list' object has no attribute 'destroy'

# Generating Map

https://carla.readthedocs.io/en/latest/tuto_G_openstreetmap/

https://www.openstreetmap.org/

run this from CARLA_0.9.11/PythonAPI/examples directory

https://github.com/carla-simulator/carla/issues/3415

In [2]:
import glob
import os
import sys
import random
import time
import numpy as np

try:
    sys.path.append(glob.glob('carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

## Conversion to OpenDrive Format

In [ ]:
# Read the .osm data
f = open("dhaka.osm", 'r') 
osm_data = f.read()
f.close()
print("Map read successful")

# Define the desired settings. In this case, default values.
settings = carla.Osm2OdrSettings()
# Convert to .xodr
xodr_data = carla.Osm2Odr.convert(osm_data, settings)
print("Conversion completepytho")

# save opendrive file
f = open("dhaka.xodr", 'w')
f.write(xodr_data)
f.close()
print("Map saved as opendrive file")

## Importing into simulator

In [4]:
client = carla.Client('localhost', 2000)
client.set_timeout(4.0)
time.sleep(4.0)
print("Establishing Connection to Server")

Establishing Connection to Server


In [7]:
# Read the .osm data
f = open("dhaka.xodr", 'r') 
xodr_data = f.read()
f.close()
print("xodr map read successful")

print(type(xodr_data))

xodr map read successful
<class 'str'>


In [8]:
vertex_distance = 2.0  # in meters
max_road_length = 500.0 # in meters
wall_height = 0.0      # in meters
extra_width = 0.6      # in meters
world = client.generate_opendrive_world(
    xodr_data, carla.OpendriveGenerationParameters(
        vertex_distance=vertex_distance,
        max_road_length=max_road_length,
        wall_height=wall_height,
        additional_width=extra_width,
        smooth_junctions=True,
        enable_mesh_visibility=True))